<a href="https://colab.research.google.com/github/HodaMemar/Patient-Similarity-through-Representation/blob/main/NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# This is formatted as code
```

# **Preprocessing Unstructured data**

Step 1: Detect sections in Clinical Notes


In [ ]:
from google.colab import drive
drive.mount('/content/mydrive/')
!ls


In [ ]:
!pip install medspacy


In [ ]:
import pandas as pd

In [ ]:
with open("/content/example_discharge_summary.txt") as f:
    text = f.read()

In [ ]:
CATEGORY='Discharge summary'
HADM_ID=159647
CHARTDATE='2146/07/28'

In [ ]:
import spacy;
import medspacy
# Option 1: Load default
nlp = medspacy.load()



In [ ]:
sectionizer = nlp.add_pipe("medspacy_sectionizer")

In [ ]:
nlp.pipe_names

In [ ]:
doc = nlp(text)

In [ ]:
len(doc._.sections)

In [ ]:
doc._.section_bodies[3]


In [ ]:
doc._.section_titles[3]

In [ ]:
doc._.section_categories[3]

In [ ]:
doc._.section_spans[3]

In [ ]:

import re
def preprocess(x):
    y=re.sub('\[(.*?)\]','',x) #remove de-identified brackets
    y=re.sub('[0-9]+\.','',y) #remove 1.2. since the segmenter segments based on this
    y=re.sub('dr\.','doctor',y)
    y=re.sub('m\.d\.','md',y)
    y=re.sub('admission date:','',y)
    y=re.sub('Admission Date:','',y)
    y=re.sub('discharge date:','',y)
    y=re.sub('Discharge Date:','',y)
    y=re.sub('Date of Birth:','',y)
    y=re.sub('date of birth:','',y)
    y=re.sub('--|__|==','',y)
    y = y.strip()
    return y

In [ ]:

ls_notes=[]
for j in range(len(doc._.sections)):
          mystr=str(doc._.section_spans[j])
          mystr_Wothout_line=preprocess(mystr)
          ls_notes.append([HADM_ID,CHARTDATE,CATEGORY,mystr_Wothout_line,doc._.section_categories[j]])

df2=pd.DataFrame(ls_notes,columns=["HADM_ID",'CHARTDATE','CATEGORY',"body_span_Wothout_line",'category'])

df2

Step 2 : Finding CUI from each section text

In [ ]:
!pip install scispacy

In [ ]:
!pip install negspacy

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
CATEGORY='Discharge summary'
HADM_ID=159647
CHARTDATE='2146/07/28'

In [ ]:
import re

import scispacy
import spacy

from negspacy.negation import Negex
from negspacy.termsets import termset
from scispacy.linking import EntityLinker
import pandas as pd

#nlp = spacy.load("en_ner_bc5cdr_md")
nlp = spacy.load("en_core_web_sm")
#
ts = termset("en_clinical")
nlp.add_pipe("negex", config={"neg_termset": ts.get_patterns()})
nlp.add_pipe("scispacy_linker", config={"linker_name": "umls", "max_entities_per_mention": 1})

linker = nlp.get_pipe("scispacy_linker")
nlp.pipe_names

In [ ]:
nlp.pipe_names

In [ ]:
text='Pt. is a 76 yo female with a history of ESRD on HD, CAD,atrial fibrillation, and DM2 who presents with a 1 day history of 3 episodes of nonbloody, nonbilious, clear emesis and 1  episode of nonbloody diarrhea.'

In [ ]:
d = {'HADM_ID':159647, 'CHARTDATE': '2146/07/28','body_span_Wothout_line':text, 'category':'past_medical_history'}
df = pd.DataFrame(data=d,index=[0])
df

In [ ]:
list_Cui=[]
list_Exception=[]



for i in range(len(df)):
            text= str(df.iloc[i]['body_span_Wothout_line'])
            HADM_ID	=df.iloc[i]['HADM_ID']
            CHARTDATE	=df.iloc[i]['CHARTDATE']

            category_Inner=	df.iloc[i]['category']

            doc = nlp(text)


            print("\n")
            for entity in doc.ents:

                if(len(entity._.kb_ents)>0):
                    first_cuid = entity._.kb_ents[0][0]
                    kb_entry = linker.kb.cui_to_entity[first_cuid]
                    list_Cui.append([HADM_ID,CHARTDATE,category_Inner,entity._.negex,entity.text,
                                     first_cuid, kb_entry.canonical_name,entity.label_  ])
                else:
                    list_Exception.append([HADM_ID,CHARTDATE,category_Inner,entity.text])
                    continue



df_cui=pd.DataFrame(list_Cui,columns=['HADM_ID','CHARTDATE','category_Inner','negex','entity_text',
                                      'first_cuid', 'canonical_name','label'
                                          ])
df_cui